In [ ]:
%matplotlib inline
import os
import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
ROOT_DIR = os.path.abspath("../")
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log
import script
MODEL_DIR = os.path.join(ROOT_DIR, "logs")
MODEL_WEIGHTS_PATH = ROOT_DIR +"/mask_rcnn_coco.h5"
import pandas as pd
import time

from sklearn.metrics import average_precision_score, precision_recall_curve
from sklearn.metrics import auc, plot_precision_recall_curve, multilabel_confusion_matrix, classification_report, confusion_matrix

In [ ]:
config = script.CustomConfig()
BEAGLE_DIR = ROOT_DIR+"/fine-tune-MaskRcnn/glitch"

In [ ]:
# Override the training configurations with a few
# changes for inferencing.
class InferenceConfig(config.__class__):
    # Run detection on one image at a time
    GPU_COUNT = 1
    BACKBONE = 'resnet101'
    IMAGES_PER_GPU = 1
    DETECTION_MIN_CONFIDENCE = 0.9

config = InferenceConfig()
config.display()

In [ ]:
# set target device
DEVICE = "/gpu:0"  # /cpu:0 or /gpu:0

In [ ]:
def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [ ]:
dataset = beagle.CustomDataset()
dataset.load_custom(GLITCH_DIR, "val")

# Must call before using the dataset
dataset.prepare()

print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))

In [ ]:
# Create model in inference mode
with tf.device(DEVICE):
    model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR,
                              config=config)

In [ ]:
weights_path = "../logs/WEIGHTFILE.h5"

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

In [ ]:
TP, TN, FP, FN = 0, 0, 0, 0
y_true2 = []
y_pred2 = []

y_true = []
y_scores = []
counts = [0, 0]

elapsed = 0


# image_ids = np.random.choice(dataset.image_ids, 80)
for image_id in dataset.image_ids:
    
    start = time.time()
    
    
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
            modellib.load_image_gt(dataset, config, image_id, use_mini_mask=False)
    results = model.detect([image], verbose=1)
    ax = get_ax(1)
    r = results[0]
    
    visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                                    dataset.class_names, r['scores'], ax=ax,
                                    title="Predictions")
    end = time.time()
    elapsed = elapsed + (end-start)
    
    totalimg = np.zeros((1024, 1024))
    for i in range(r['masks'].shape[-1]):
        mask = r['masks'][:, :, i]
        totalimg = np.add(totalimg, mask)
        image[mask] = 255
        image[~mask] = 0
        unique, counts = np.unique(image, return_counts=True)
        
        if not np.all((gt_mask == 0)): #if not completely normal
            groundtruth = np.reshape(gt_mask, (-1, gt_mask.shape[-1])).astype(np.float32)
        #         predicted = float(np.count_nonzero(totalimg))

            print(np.shape(gt_mask))
            print(np.shape(totalimg))

            gtruth = gt_mask[:,:,0]

            for i in range(0, gt_mask.shape[-1]-1):
                print(i)
                out = np.nonzero((totalimg!=0) & (gt_mask[:,:,i]!=0))

            if len(counts) == 1:
                y_scores.append(0)
            else:
                y_scores.append(2 * np.count_nonzero(out) / (float(np.count_nonzero(totalimg)) + groundtruth.sum()))            
    
    if (len(gt_class_id) == 0):
        y_true.append(0)
    else:
        y_true.append(1)

    predictionVector = np.zeros(4)
    
    for i in range(len(r['class_ids'])):
        for c in range(1,4): # c = 1:missing 2:low_res 3:stretched
            if r['class_ids'][i] == c and r['scores'][i] > predictionVector[c]:
                predictionVector[c] = r['scores'][i]
    if np.array_equal(predictionVector, np.zeros(4)):
        predictionVector[0] = 1.0

    groundtruthVector =  np.zeros(4)
    
    for p in gt_class_id:
        groundtruthVector[p] = 1.0
    if np.array_equal(groundtruthVector, np.zeros(4)):
        groundtruthVector[0] = 1.0

    print(predictionVector, groundtruthVector)
    
    if predictionVector[0] == groundtruthVector[0]:
        if groundtruthVector[0] == 0:
            TN += 1
        else:
            TP += 1
    else:
        if groundtruthVector[0] == 0:
            FP += 1
        else:
            FP += 1
    
    y_true2.append(groundtruthVector)
    y_pred2.append(np.round(predictionVector))


print(elapsed)


In [ ]:
print(TP, TN, FP, FN) 

precision = TP / (TP + FP)
recall = TP / (TP + FN)
f1score = (2 * precision * recall) / (precision + recall)

print(precision, recall)
print(f1score) 

y_pred2 = np.round(y_pred2)

# TN FN
# FP TP

cm = multilabel_confusion_matrix(y_true2, y_pred2)
print(cm)
print(classification_report(y_true2,y_pred2))


In [ ]:
print(y_true)
print(y_scores)

precision, recall, thresholds = precision_recall_curve(y_true, y_scores)

# print(precision)
# print(recall)

plt.figure()
plt.axis('equal')
plt.plot(recall, precision, lw=2, color='navy')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.grid()
plt.title('Precision-Recall AUC={0:0.2f}'.format(average_precision_score(
        y_true, y_scores))) 